In [1]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import glob
from sklearn.model_selection import train_test_split
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [2]:
image_list = []
genres = []
for filename in glob.glob('../images_original/*/*.png'): #assuming png
    im = np.array(Image.open(filename).convert('L'))/255
    genre = filename.split('/')[2]
    
    genres.append(genre)
    image_list.append(im)

In [3]:
unique_genres = list(np.unique(genres))
unique_genres

['blues',
 'classical',
 'country',
 'disco',
 'hiphop',
 'jazz',
 'metal',
 'pop',
 'reggae',
 'rock']

In [4]:
def map_genre_to_onehot(genre_str):
    '''
    Genre str: String containing genre name
    Returns: one hot encoding of genre string given list of classes
    '''
    ohe = np.zeros(shape=(10,))
    genre_idx = unique_genres.index(genre_str)
    ohe[genre_idx] = 1
    return ohe

In [5]:
def map_genre_to_number(genre_str):
    genre_idx = unique_genres.index(genre_str)
    return genre_idx

In [6]:
genres = pd.Series(genres)
genres_nums = genres.map(map_genre_to_number).values
genres_nums

array([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0,

In [7]:
y = to_categorical(genres_nums)
y.shape

(999, 10)

In [8]:
genres = pd.Series(genres)
genres_onehot = genres.map(map_genre_to_onehot)

In [9]:
X = np.asarray(image_list).reshape(999,288,432,1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
#create model
model = tf.keras.Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(288,432,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Epoch 1/3
20/25 [=======================>......] - ETA: 41s - loss: 16.3660 - accuracy: 0.1203